In [90]:
from qiskit.providers.ibmq.job.ibmqjob import ibmqbackend

In [87]:
def different_borrow_XNOR(bitA, bitB, Borrow):
    qreg_q = QuantumRegister(4, 'q')
    creg_dbx = ClassicalRegister(3, 'dbx')
    circuit = QuantumCircuit(qreg_q, creg_dbx)
    
    # add the 2bits value to the subtractor circuit as (bitA - bitB)
    if(bitA == 1):circuit.x(qreg_q[0])
    if(bitB == 1):circuit.x(qreg_q[1])
    if(Borrow == 1):circuit.x(qreg_q[3])
    circuit.barrier(qreg_q)
    
    # constracting the G^3(132) gate
    circuit.cx(qreg_q[1], qreg_q[2])
    circuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])
    circuit.cx(qreg_q[0], qreg_q[2])
    circuit.x(qreg_q[2])
    circuit.barrier(qreg_q)
    # constracting the G^3(231) gate
    circuit.ccx(qreg_q[2], qreg_q[3], qreg_q[1])
    circuit.cx(qreg_q[2], qreg_q[3])
    circuit.x(qreg_q[3])
    circuit.barrier(qreg_q)
    
    # measuring the different, Borrow and XNOR qubit values
    circuit.measure(qreg_q[1], creg_dbx[1])
    circuit.measure(qreg_q[2], creg_dbx[2])
    circuit.measure(qreg_q[3], creg_dbx[0])
    
    # runnim the circuit on simulator and return list of the dbx
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circuit, backend = backend, shots = 1024)  # any number of shots can be good in this case, i used the stander 1024 
    result = job.result()
    counts = result.get_counts()
    plot_histogram(counts)
    k, v = counts.popitem()
    return list(map(int,str(k)))
    

In [88]:
def find_the_largest_number(A,B):
    
    # convert both numbers to the its binary representation and make them the both size list
    a = bin(A).replace("-0b", "1").replace("0b", "0")
    a = list(map(int,str(a)))
    b = bin(B).replace("-0b", "1").replace("0b", "0")
    b = list(map(int,str(b)))
    if(len(a) > len(b)):
      for i in range( len(a) - len(b) ):
        b.insert(1,0)
    elif(len(a)<len(b)):
       for i in range( len(b) - len(a) ):
        a.insert(1,0)

    # first will check the sign bits with the same circuit by the XNOR output
    sign_bit = different_borrow_XNOR(a[0],b[0],0)
    if(sign_bit[0] == 0):
      if(sign_bit[1] == 1):return A 
      else: return B

    # now we sure both numbers are positive or negative we load both numbers bit by bit to our full Subtractor circuit
    borrow = 0
    for i in reversed(range(1,len(a))):
      dbx = different_borrow_XNOR(a[i],b[i],borrow)
      borrow = dbx[1]
    
    # after subtracting the 2 numbers we have the last dbx with the last borrow value representing the value sign
    # if both numbers were positve & ( A > B ) the borrow would be 0
    # if both numbers were negative & ( A > B ) the borrow would be 1
    if( dbx[1] == 0 ):
      if( a[0] == 0 ): return A
      else: return B
    elif( a[0] == 0 ): return B
    else: return A
    


In [89]:
print(find_the_largest_number(5,-6))
print(find_the_largest_number(-25,-12))
print(find_the_largest_number(1024,256))

5
-12
1024
